# 5.7. Analiza cech istotnych dla modelu

Regresja logistyczna, podobnie jak regresja liniowa, pozwala określić wagę poszczególnych cech, ponieważ każdej zmiennej jest przypisywana waga. Kolejny raz zachodzi jednak problem, że same wagi są zależne od jednostki danej zmiennej i nie można na nie patrzeć jak na bezpośrednie współczynniki ważności. Dróg interpretacji tych wag jest kilka - ja preferuję wcześniejszą normalizację danych i z tego podejścia skorzystamy.

Wybraliśmy już cechy, które pozwalają nam osiągnąć najlepsze rezultaty. W związku z tym, wczytamy sobie wyłącznie zredukowany zbiór danych.

In [1]:
import pandas as pd

In [3]:
credit_cards_df = pd \
    .read_parquet("../data/credit-cards-reduced.parquet")
credit_cards_df.sample(n=5).T

ID,20306,11291,909,29663,14541
AGE,62.000000,43.000000,27.000000,31.000000,62.000000
PAY_1,0.000000,0.000000,0.000000,0.000000,0.000000
PAY_2,0.000000,0.000000,0.000000,0.000000,0.000000
PAY_3,0.000000,0.000000,0.000000,0.000000,0.000000
PAY_4,0.000000,0.000000,0.000000,0.000000,0.000000
PAY_5,0.000000,0.000000,0.000000,0.000000,0.000000
PAY_6,2.000000,0.000000,0.000000,0.000000,0.000000
PAY_OVERDUE_COUNT,1.000000,0.000000,0.000000,0.000000,0.000000
WEIGHTED_PAYMENT_HISTORY,0.333333,0.000000,0.000000,0.000000,0.000000
AVG_PAY_AMT,2686.500000,907.166667,1416.666667,1833.333333,2741.833333


## Modelowanie danych ze wstępnym skalowaniem

Zdefiniujmy łańcuch przetwarzania danych dla naszego zbioru, tym razem ze skalowaniem zmiennych. Nie będziemy przeprowadzać walidacji krzyżowej, ponieważ już sprawdziliśmy możliwości predykcyjne naszego modelu i wiemy, że powinien on być w stanie działać na akceptowalnym dla nas poziomie.

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

In [9]:
X = credit_cards_df.drop(columns="DEFAULT")
y = credit_cards_df["DEFAULT"]

In [10]:
pipeline = Pipeline(steps=[
    ("scaling", MinMaxScaler()),
    ("classification", LogisticRegression(penalty="none",
                                          max_iter=10000))
])
pipeline.fit(X, y)

Pipeline(memory=None,
         steps=[('scaling', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('classification',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=10000,
                                    multi_class='auto', n_jobs=None,
                                    penalty='none', random_state=None,
                                    solver='lbfgs', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

In [19]:
pd.DataFrame({
    "coef": pipeline.steps[1][1].coef_[0]
}, index=X.columns)

,coef
AGE,0.108524
PAY_1,2.877750
PAY_2,-0.680596
PAY_3,-0.586397
PAY_4,-0.850824
PAY_5,-0.543092
PAY_6,-0.516250
PAY_OVERDUE_COUNT,2.896217
WEIGHTED_PAYMENT_HISTORY,0.923446
AVG_PAY_AMT,-17.839660


Z czystej ciekawości możemy także sprawdzić, jak model jest w stanie przewidywać zbiór uczący.

In [21]:
from sklearn.metrics import classification_report

In [24]:
print(
    classification_report(y, pipeline.predict(X))
)

              precision    recall  f1-score   support

           0       0.83      0.96      0.89     23364
           1       0.66      0.28      0.40      6636

    accuracy                           0.81     30000
   macro avg       0.74      0.62      0.64     30000
weighted avg       0.79      0.81      0.78     30000

